In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from IPython.display import Javascript
import getpass

In [4]:
password = None
if password is None:
    password = getpass.getpass()

In [3]:
!echo $password | sudo -S mount -t tmpfs -o size=15g tmpfs /mnt/tmpfs/
!cp -r ~/Documents/distros /mnt/tmpfs/

[sudo] password for fbdtemme: Sorry, try again.
[sudo] password for fbdtemme: 


^C


In [5]:
pwd = "/mnt/tmpfs"
max_threads = 4
iterations = 5
target = 'distros'

In [6]:
fs = !du -b $pwd/$target | grep -o "[0-9]*"
file_size = int(fs[0])

In [7]:
file_size

13744259172

In [8]:
# Disable cpu frequency scaling
!echo $password | sudo -S cpupower frequency-set -g performance

[sudo] password for fbdtemme: Setting cpu: 0
Setting cpu: 1
Setting cpu: 2
Setting cpu: 3
Setting cpu: 4
Setting cpu: 5
Setting cpu: 6
Setting cpu: 7


In [9]:
def parse_real_time(data: str):
    results = []
    match = re.search(R"real\s(\d+)m([\d.]+)s", data)
    minutes = float(match.group(1))
    seconds = float(match.group(2))
    return minutes * 60 + seconds

def parse_real_time_sequence(data: str):
    results = []
    for match in re.findall(R"real\s(\d+)m([\d.]+)s", data):
        print(match[0])
        print(match[1])
        minutes = float(match[0])
        seconds = float(match[1])
        results.append(minutes * 60 + seconds)
    return results

In [10]:
data = {}

In [11]:
def run_benchmark(name: str, use_threads:bool = True):
    duration = np.ndarray(shape=(iterations, max_threads))
    
    if use_threads:
        for t in range(1, max_threads+1):
            for i in range(0, iterations):
                result = !./benchmark.sh "{name}" "{pwd}" "{target}" "{t}"
                result_str = '\n'.join(result)
                print(f"-- iteration {i+1} | threads {t}\n{result_str}")
                duration[i, t-1] = parse_real_time(result_str)
    else:
        for i in range(0, iterations):
            t = 1
            result = !./benchmark.sh "{name}" "{pwd}" "{target}" "{t}"
            result_str = '\n'.join(result)
            print(f"-- iteration {i+1} | threads {t}\n{result_str}")
            duration[i, :] = parse_real_time(result_str)

    speed = file_size / duration
    mean = np.mean(speed, axis=0)
    std  = np.std(speed, axis=0)
    return (mean, std)

### mktorrent

In [12]:
mktorrent_data = run_benchmark("mktorrent")

-- iteration 1 | threads 1

real	0m17.691s
user	0m17.348s
sys	0m4.444s

-- iteration 2 | threads 1

real	0m16.449s
user	0m16.378s
sys	0m3.764s

-- iteration 3 | threads 1

real	0m16.565s
user	0m16.529s
sys	0m4.142s

-- iteration 4 | threads 1

real	0m16.707s
user	0m16.646s
sys	0m4.399s

-- iteration 5 | threads 1

real	0m17.004s
user	0m16.854s
sys	0m4.812s

-- iteration 1 | threads 2

real	0m9.657s
user	0m18.695s
sys	0m5.659s

-- iteration 2 | threads 2

real	0m9.347s
user	0m18.429s
sys	0m5.626s

-- iteration 3 | threads 2

real	0m9.406s
user	0m18.443s
sys	0m5.641s

-- iteration 4 | threads 2

real	0m9.292s
user	0m18.308s
sys	0m5.459s

-- iteration 5 | threads 2

real	0m9.280s
user	0m18.238s
sys	0m5.521s

-- iteration 1 | threads 3

real	0m6.525s
user	0m19.217s
sys	0m5.316s

-- iteration 2 | threads 3

real	0m6.581s
user	0m19.469s
sys	0m5.310s

-- iteration 3 | threads 3

real	0m6.563s
user	0m19.314s
sys	0m5.341s

-- iteration 4 | threads 3

real	0m6.774s
user	0m19.926s
sys	0m5.345s

-

In [13]:
torrenttools_openssl_data = run_benchmark("torrenttools_openssl")

-- iteration 1 | threads 1

real	0m18.022s
user	0m17.741s
sys	0m4.572s

-- iteration 2 | threads 1

real	0m18.018s
user	0m17.970s
sys	0m4.645s

-- iteration 3 | threads 1

real	0m18.018s
user	0m17.336s
sys	0m4.120s

-- iteration 4 | threads 1

real	0m18.018s
user	0m17.573s
sys	0m4.316s

-- iteration 5 | threads 1

real	0m18.019s
user	0m18.029s
sys	0m4.588s

-- iteration 1 | threads 2

real	0m10.019s
user	0m18.254s
sys	0m4.737s

-- iteration 2 | threads 2

real	0m10.020s
user	0m18.224s
sys	0m4.553s

-- iteration 3 | threads 2

real	0m10.020s
user	0m18.015s
sys	0m4.496s

-- iteration 4 | threads 2

real	0m10.022s
user	0m19.211s
sys	0m5.284s

-- iteration 5 | threads 2

real	0m10.019s
user	0m19.605s
sys	0m5.482s

-- iteration 1 | threads 3

real	0m8.030s
user	0m21.087s
sys	0m5.462s

-- iteration 2 | threads 3

real	0m8.030s
user	0m20.915s
sys	0m5.433s

-- iteration 3 | threads 3

real	0m7.019s
user	0m19.704s
sys	0m5.357s

-- iteration 4 | threads 3

real	0m8.018s
user	0m21.463s
sys	0m5.46

In [14]:
torrenttools_isal_data = run_benchmark("torrenttools_isal")

-- iteration 1 | threads 1

real	0m8.056s
user	0m7.082s
sys	0m4.005s

-- iteration 2 | threads 1

real	0m8.034s
user	0m6.931s
sys	0m4.061s

-- iteration 3 | threads 1

real	0m8.033s
user	0m7.279s
sys	0m4.213s

-- iteration 4 | threads 1

real	0m8.038s
user	0m7.405s
sys	0m4.275s

-- iteration 5 | threads 1

real	0m8.036s
user	0m7.327s
sys	0m4.300s

-- iteration 1 | threads 2

real	0m5.039s
user	0m8.855s
sys	0m4.817s

-- iteration 2 | threads 2

real	0m5.045s
user	0m8.703s
sys	0m4.752s

-- iteration 3 | threads 2

real	0m5.043s
user	0m8.858s
sys	0m4.869s

-- iteration 4 | threads 2

real	0m6.049s
user	0m8.799s
sys	0m4.907s

-- iteration 5 | threads 2

real	0m6.048s
user	0m8.937s
sys	0m4.909s

-- iteration 1 | threads 3

real	0m6.037s
user	0m9.880s
sys	0m5.065s

-- iteration 2 | threads 3

real	0m6.049s
user	0m9.789s
sys	0m5.000s

-- iteration 3 | threads 3

real	0m5.048s
user	0m9.704s
sys	0m4.863s

-- iteration 4 | threads 3

real	0m5.047s
user	0m9.804s
sys	0m4.858s

-- iteration 5 | thr

In [15]:
imdl_data = run_benchmark("imdl", use_threads=False)

^C
-- iteration 1 | threads 1



AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
dottorrent_cli_data = run_benchmark("dottorrent-cli", use_threads=False)

In [ ]:
pyrocore_data = run_benchmark("pyrocore", use_threads=False)

In [ ]:
transmission_create_data = run_benchmark("transmission-create", use_threads=False)

In [ ]:
py3createtorrent_data = run_benchmark("py3createtorrent", use_threads=False)

In [ ]:
torf_data = run_benchmark("torf-cli")

In [ ]:
benchmark_data = {
    "mktorrent": mktorrent_data, 
    "imdl": imdl_data,
    "dottorrent-cli": dottorrent_cli_data,
    "pyrocore": pyrocore_data,
    "transmission-create": transmission_create_data,
    "py3createtorrent" : py3createtorrent_data,
    "torf-cli": torf_data,
    "torrenttools (OpenSSL)": torrenttools_openssl_data,
    "torrenttools (ISA-L)": torrenttools_isal_data,
}

In [ ]:
programs = list(benchmark_data.keys())
threads = list(range(1, max_threads+1))
index = pd.MultiIndex.from_product((threads, programs), names=["threads", "programs"])

speed_mean_data = {}
speed_stddev_data = {}
for program, data in benchmark_data.items():
    speed_mean_data[program] = data[0]
    speed_stddev_data[program] = data[1]
    

speed_mean = pd.DataFrame(speed_mean_data).stack()
speed_stddev = pd.DataFrame(speed_stddev_data).stack()

In [ ]:
df = pd.DataFrame()
df["speed_mean"], df["speed_stddev"] = speed_mean / 1e6, speed_stddev / 1e6
df.index = index

In [36]:
# df.to_csv("benchmark.csv")
df = pd.read_csv("benchmark.csv", header=1)
df

ParserError: Passed header=1 but only 1 lines in file

In [35]:
import matplotlib.cm
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

programs = df.index.unique(level=1)
threads = df.index.unique(level=0)
programs_by_max_speed = {}

for program in programs:
    sm = df.loc[df.index.get_level_values(1) == program, "speed_mean"]
    programs_by_max_speed[program] = np.max(sm)

sorted_programs = sorted(programs_by_max_speed.keys(), key=lambda x : programs_by_max_speed[x], reverse=True)

for idx, program in enumerate(sorted_programs):
    sm = df.loc[df.index.get_level_values(1) == program, "speed_mean"]
    sd = df.loc[df.index.get_level_values(1) == program, "speed_stddev"]
    ax.plot(threads, sm)
    ax.fill_between(threads, sm - sd, sm + sd, alpha=0.1)

ax.set_title("Performance")
ax.set_xlabel("# hashing threads")
ax.set_ylabel("Hash rate (MiB/s)")
ax.set_xticks(list(range(1, max_threads+1)))
ax.legend(sorted_programs, bbox_to_anchor=(1.05, 1), loc='upper left')

IndexError: Too many levels: Index has only 1 level, not 2